In [ ]:
# ------------------------------------------
# WEATHER DATA - IMPORT
# ------------------------------------------

import requests
import pandas as pd

# Global variables needed to run the function independently
#weather_time = '2024-02-19/2024-03-19'
#weather_station = 'SN68860'
#weather_elements = 'mean(air_temperature P1D),sum(precipitation_amount P1D),mean(wind_speed P1D)'
#weather_resolution = 'P1D' 

def get_met():

    # Client ID to access data from 
    client_id = 'd933f861-70f3-4d0f-adc6-b1edb5978a9e'

    # Define endpoints and parameters
    endpoint = 'https://frost.met.no/observations/v0.jsonld'
    parameters = {
        'sources': weather_station,  # Station ID for Voll weather station
        'elements': weather_elements,  # Requestion various types of eather data
        'referencetime': weather_time,  # Limiting the time frame for the data request
        'timeresolutions': weather_resolution,  # Set the resolution(granularity) of the data
    }

    # Send an HTTP GET-request
    r = requests.get(endpoint, params=parameters, auth=(client_id, ''))

    # Extract JSON-data
    json_data = r.json()

    # Check if the request was succesfull, and exit if not
    if r.status_code == 200:
        data = json_data['data']
        print('Data collected from frost.met.no!')
    else:
        print('Error! Statuscode:', r.status_code)
        print('Message:', json_data['error']['message'])
        print('Cause:', json_data['error']['reason'])
        exit()  

    # Create and set up the dataframe
    df = pd.DataFrame()
    for obs in data:
        row = pd.DataFrame(obs['observations'])
        row['referenceTime'] = obs['referenceTime']
        df = pd.concat([df, row], ignore_index=True)


    # Remove uneeded collumns  
    '''add timeoffset if we decide not to use it: , "timeOffset"'''

    columns_to_drop = ["level", "timeResolution", "timeSeriesId", "elementId", "performanceCategory", "exposureCategory", "qualityCode"]
    df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])

    # Remove time portion from 'referenceTime' (keep only 'YYYY-MM-DD')
    df["referenceTime"] = df["referenceTime"].str.split("T").str[0]

    print('There are ', df.shape[0], 'lines of data in this dataframe.\n')

    #Returns dataframe upon request
    return(df)




# *** EVERYTHING below this line is purely for local testing purposes of this specific file ***
'''Fjern # foran de to linjene under som kjører funksjonen og så printer den, for å se om det virker :)
Husk og de kommenterte variablene på toppen!'''

    # Run the function to get the dataframe with the weather data
#df=get_met()

    # Show the first few rows of the dataframe
#print(df.head())

Data collected from frost.met.no!
There are  145 lines of data in this dataframe.

   value  unit timeOffset referenceTime
0   -5.5  degC       PT0H    2024-02-19
1   -5.3  degC       PT6H    2024-02-19
2    0.0    mm      PT18H    2024-02-19
3    0.0    mm       PT6H    2024-02-19
4    0.7   m/s       PT0H    2024-02-19
